This notebook performs Gaussian Process Regression

Training Inputs $\bold{x}_T = \{x_0, x_1, ..., x_n\}$

Training Outputs $\bold{y}_T = \{y_0, y_1, ..., y_n\}$

Test Inputs $\bold{x_*} = \{x_{*_0}, x_{*_1}, ..., x_{*_n}\}$

Observation Noise: $\bold{y} = \bold{f} + \epsilon, \epsilon \sim \mathscr{N}(0,\sigma_n^2)$

Kernel Function $k(a,b) = \sigma_f^2\exp(-|a-b|^2)$

Posterior Mean: $\mu = f_0 + K_{xx}(K_{xx}+\sigma_n^2I)^{-1}(y-f_0)$

Posterior Covariance: $\Sigma = K_{xx} - K_{xx}(K_{xx}+\sigma_n^2I)^{-1}K_{xx}$

Predictive Mean: $\mu_* = f_{0_*} + K_{*x}(K_{xx}+\sigma_n^2I)^{-1}(y-f_0)$

Predictive Covariance: $\Sigma_* = K_{**} - K_{*x}(K_{xx}+\sigma_n^2I)^{-1}K_{x*}$


In [81]:
import streamlit as st
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

Data Input

In [82]:
X = np.array([1,2,2.5,3,4,5])
Y = np.array([0,3,1,8,7,2])
sigman = .1
sigmaf = 1
l = 1
f0 = np.zeros_like(X)

fig = px.scatter(x=X, y=Y)
fig.show()

In [83]:
def k(a,b, sf, l):
    return np.exp(-.5 * np.linalg.norm(a-b)**2 / l**2) * sf**2

Kxx = np.array([[k(x1,x2, sigmaf, l) for x1 in X] for x2 in X])
A = np.linalg.inv(Kxx + sigman**2 * np.eye(len(X)))
mu = f0 + Kxx @ A @ (Y - f0)
COV = Kxx - Kxx @ A @ Kxx

for x,a,b,c in zip(X, Y, mu, np.diag(COV)): print(f"X={x}, Y={a}, mu={b:.4f}, var={c:.4f}")

fig = px.scatter(x=X, y=mu)
fig.add_trace(px.line(x=X, y=Y, color_discrete_sequence=['red'], labels={'x':'X', 'y':'Y'}).data[0])

fig.show()


x_star = np.linspace(-4,10,100)
Kxs = np.transpose(np.array([[k(x1,x2, sigmaf, l) for x1 in X] for x2 in x_star]))
Kss = np.array([[k(x1,x2, sigmaf, l) for x1 in x_star] for x2 in x_star])

print(f"Kxx shape: {Kxx.shape}, Kxs shape: {Kxs.shape}, Kss shape: {Kss.shape}, A shape: {A.shape}, Y shape: {Y.shape}, f0 shape: {f0.shape}")
mu_star = np.transpose(Kxs) @ A @ Y
COV_star = Kss - np.transpose(Kxs) @ A @ Kxs
std = np.diag(COV_star)**.5

fig = px.line(x=x_star, y=mu_star, labels={'x':'X', 'y':'Y'})
fig.add_trace(px.scatter(x=X, y=Y, color_discrete_sequence=['red'], labels={'x':'X', 'y':'Y'}).data[0])
fig.add_trace(px.line(x=x_star, y=mu_star+std).data[0])
fig.add_trace(px.line(x=x_star, y=mu_star-std).data[0])
fig.show()


X=1.0, Y=0, mu=0.2038, var=0.0098
X=2.0, Y=3, mu=1.9526, var=0.0084
X=2.5, Y=1, mu=2.8204, var=0.0062
X=3.0, Y=8, mu=6.7852, var=0.0081
X=4.0, Y=7, mu=7.2628, var=0.0096
X=5.0, Y=2, mu=1.9174, var=0.0098


Kxx shape: (6, 6), Kxs shape: (6, 100), Kss shape: (100, 100), A shape: (6, 6), Y shape: (6,), f0 shape: (6,)
